In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt

from multivariable_analysis import concatenated_df

In [ ]:
cpp_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/cpp_df.csv'
ptio2_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/ptio2_df.csv'
filtered_ptio2_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/ptio2_df_filtered.csv'
temperature_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/temperature_df.csv'
lpr_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/lpr_df.csv'
hr_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/hr_df.csv'
etco2_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/etco2_df.csv'
ci_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/ci_df.csv'
prx_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/prx_df.csv'
drug_administration_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/drug_administrations.xlsx'
registry_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_registry_kssg_post_hoc_modified.xlsx'
paco2_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/pdms_data/joined_aBGA.csv'
mainstream_etco2_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/neurocrit_fever/data/PDMS_data/joined_etCO2.csv'

In [ ]:
exclude_short_infusions = True
use_filtered_ptio2 = True

In [ ]:
cpp_df = pd.read_csv(cpp_path)
if use_filtered_ptio2:
    ptio2_df = pd.read_csv(filtered_ptio2_path)
else:
    ptio2_df = pd.read_csv(ptio2_path)
temperature_df = pd.read_csv(temperature_path)
lpr_df = pd.read_csv(lpr_path)
hr_df = pd.read_csv(hr_path)
prx_df = pd.read_csv(prx_path)
drug_administration_df = pd.read_excel(drug_administration_path)

In [ ]:
registry_df = pd.read_excel(registry_path)
paco2_df = pd.read_csv(paco2_path, sep=';')
mainstream_etco2_df = pd.read_csv(mainstream_etco2_path, sep=';')

In [ ]:
paco2_df = pd.merge(paco2_df, registry_df[['manual_mrn', 'Pat. Nr.']], left_on='FallNr', right_on='manual_mrn',
                    how='left')
paco2_df.drop(columns=['manual_mrn'], inplace=True)
paco2_df.rename(columns={'Pat. Nr.': 'pat_nr'}, inplace=True)
paco2_df.rename(columns={'Zeitpunkt_aBGA': 'datetime'}, inplace=True)
paco2_df['pCO2_mmHg'] = paco2_df['pCO2'] * 7.50062

mainstream_etco2_df = pd.merge(mainstream_etco2_df, registry_df[['manual_mrn', 'Pat. Nr.']], left_on='FallNr', right_on='manual_mrn', how='left')
mainstream_etco2_df.drop(columns=['manual_mrn'], inplace=True)
mainstream_etco2_df.rename(columns={'Pat. Nr.':'pat_nr'}, inplace=True)
mainstream_etco2_df.rename(columns={'Zeitpunkt_etCO2':'datetime'}, inplace=True)

In [ ]:
drug_administration_df = drug_administration_df[drug_administration_df.monitored]

n_patients_before = drug_administration_df['pat_nr'].nunique()
# print patients with exclusion criterium
print(f'Excluding {drug_administration_df[~pd.isna(drug_administration_df["further_exclusion_criterium"])].shape[0]} infusions with {drug_administration_df[~pd.isna(drug_administration_df["further_exclusion_criterium"])]["further_exclusion_criterium"].nunique()} different further exclusion criteria')
# exclude if further_exclusion_criterium is not Nan
drug_administration_df = drug_administration_df[pd.isna(drug_administration_df['further_exclusion_criterium'])]
# print number of patients excluded
print(f'Excluding {n_patients_before - drug_administration_df["pat_nr"].nunique()} patients with further exclusion criterium')

if exclude_short_infusions:
    n_patients_before = drug_administration_df['pat_nr'].nunique()
    drug_administration_df['infusion_duration'] = (pd.to_datetime(drug_administration_df['drug_end']) - pd.to_datetime(drug_administration_df['drug_start'])).dt.total_seconds() / 3600
    print(f'Excluding {drug_administration_df[drug_administration_df["infusion_duration"] <= 1].shape[0]} infusions with duration <= 1h')
    drug_administration_df = drug_administration_df[drug_administration_df['infusion_duration'] > 1]
    print(f'Excluding {n_patients_before - drug_administration_df["pat_nr"].nunique()} patients with infusions with duration <= 1h')

In [ ]:
for var_df in [ptio2_df, cpp_df, temperature_df, lpr_df, hr_df, paco2_df, prx_df, mainstream_etco2_df]:
    var_df['datetime'] = pd.to_datetime(var_df['datetime'])

In [ ]:
# for every drug administration extract data from -xh to +xh around start
time_window_post = 12
time_window_pre = 12

associated_ptio2_df = pd.DataFrame()
associated_cpp_df = pd.DataFrame()
associated_temperature_df = pd.DataFrame()
associated_hr_df = pd.DataFrame()
associated_lpr_df = pd.DataFrame()
associated_ci_df = pd.DataFrame()
associated_prx_df = pd.DataFrame()
associated_etco2_df = pd.DataFrame()
associated_paco2_df = pd.DataFrame()
associated_mainstream_etco2_df = pd.DataFrame()

for index, row in drug_administration_df.iterrows():
    lower_bound = row['drug_start'] - pd.to_timedelta(time_window_pre, unit='h')
    upper_bound = row['drug_start'] + pd.to_timedelta(time_window_post, unit='h')
    instance_associated_ptio2_df = ptio2_df[(ptio2_df['pat_nr'] == row['pat_nr'])
                                            & (ptio2_df['datetime'] >= lower_bound) 
                                            & (ptio2_df['datetime'] <= upper_bound)]
    instance_associated_ptio2_df['drug_start'] = row['drug_start']
    instance_associated_ptio2_df['relative_datetime'] = (instance_associated_ptio2_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_ptio2_df = pd.concat([associated_ptio2_df, instance_associated_ptio2_df])

    instance_associated_cpp_df = cpp_df[(cpp_df['pat_nr'] == row['pat_nr'])
                                        & (cpp_df['datetime'] >= lower_bound) 
                                        & (cpp_df['datetime'] <= upper_bound)]  
    instance_associated_cpp_df['drug_start'] = row['drug_start']
    instance_associated_cpp_df['relative_datetime'] = (instance_associated_cpp_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_cpp_df = pd.concat([associated_cpp_df, instance_associated_cpp_df])

    instance_associated_temperature_df = temperature_df[(temperature_df['pat_nr'] == row['pat_nr'])
                                        & (temperature_df['datetime'] >= lower_bound)
                                        & (temperature_df['datetime'] <= upper_bound)]
    instance_associated_temperature_df['drug_start'] = row['drug_start']
    instance_associated_temperature_df['relative_datetime'] = (instance_associated_temperature_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_temperature_df = pd.concat([associated_temperature_df, instance_associated_temperature_df])

    instance_associated_hr_df = hr_df[(hr_df['pat_nr'] == row['pat_nr'])
                                        & (hr_df['datetime'] >= lower_bound)   
                                        & (hr_df['datetime'] <= upper_bound)]
    instance_associated_hr_df['drug_start'] = row['drug_start']
    instance_associated_hr_df['relative_datetime'] = (instance_associated_hr_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_hr_df = pd.concat([associated_hr_df, instance_associated_hr_df])

    instance_associated_lpr_df = lpr_df[(lpr_df['pat_nr'] == row['pat_nr'])
                                        & (lpr_df['datetime'] >= lower_bound)
                                        & (lpr_df['datetime'] <= upper_bound)]
    instance_associated_lpr_df['drug_start'] = row['drug_start']
    instance_associated_lpr_df['relative_datetime'] = (instance_associated_lpr_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_lpr_df = pd.concat([associated_lpr_df, instance_associated_lpr_df])
    
    instance_associated_paco2_df = paco2_df[(paco2_df['pat_nr'] == row['pat_nr'])
                                        & (paco2_df['datetime'] >= lower_bound)
                                        & (paco2_df['datetime'] <= upper_bound)]
    instance_associated_paco2_df['drug_start'] = row['drug_start']
    instance_associated_paco2_df['relative_datetime'] = (instance_associated_paco2_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_paco2_df = pd.concat([associated_paco2_df, instance_associated_paco2_df])
    
    instance_associated_prx_df = prx_df[(prx_df['pat_nr'] == row['pat_nr'])
                                    & (prx_df['datetime'] >= lower_bound)
                                    & (prx_df['datetime'] <= upper_bound)]
    instance_associated_prx_df['drug_start'] = row['drug_start']
    instance_associated_prx_df['relative_datetime'] = (instance_associated_prx_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_prx_df = pd.concat([associated_prx_df, instance_associated_prx_df])
    
    instance_associated_mainstream_etco2_df = mainstream_etco2_df[(mainstream_etco2_df['pat_nr'] == row['pat_nr'])
                                        & (mainstream_etco2_df['datetime'] >= lower_bound)
                                        & (mainstream_etco2_df['datetime'] <= upper_bound)]
    instance_associated_mainstream_etco2_df['drug_start'] = row['drug_start']
    instance_associated_mainstream_etco2_df['relative_datetime'] = (instance_associated_mainstream_etco2_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_mainstream_etco2_df = pd.concat([associated_mainstream_etco2_df, instance_associated_mainstream_etco2_df])



In [ ]:
allowed_ptio2_range = [0, 200]
allowed_cpp_range = [0, 200]
allowed_temperature_range = [30, 45]
allowed_hr_range = [0, 300]
allowed_lpr_range = [0, 100]
allowed_etco2_range = [0, 100]
allowed_paco2_range = [0.5, 20]  # in kpa

# drop values outside of allowed range
associated_ptio2_df = associated_ptio2_df[
    (associated_ptio2_df['ptio2'] >= allowed_ptio2_range[0]) & (associated_ptio2_df['ptio2'] <= allowed_ptio2_range[1])]
associated_cpp_df = associated_cpp_df[
    (associated_cpp_df['cpp'] >= allowed_cpp_range[0]) & (associated_cpp_df['cpp'] <= allowed_cpp_range[1])]
associated_temperature_df = associated_temperature_df[
    (associated_temperature_df['temperature'] >= allowed_temperature_range[0]) & (
                associated_temperature_df['temperature'] <= allowed_temperature_range[1])]
associated_hr_df = associated_hr_df[
    (associated_hr_df['hr'] >= allowed_hr_range[0]) & (associated_hr_df['hr'] <= allowed_hr_range[1])]
associated_lpr_df = associated_lpr_df[
    (associated_lpr_df['lpr'] >= allowed_lpr_range[0]) & (associated_lpr_df['lpr'] <= allowed_lpr_range[1])]
# associated_etco2_df = associated_etco2_df[
#     (associated_etco2_df['etco2'] >= allowed_etco2_range[0]) & (associated_etco2_df['etco2'] <= allowed_etco2_range[1])]
associated_paco2_df = associated_paco2_df[
    (associated_paco2_df['pCO2'] >= allowed_paco2_range[0]) & (associated_paco2_df['pCO2'] <= allowed_paco2_range[1])]
associated_mainstream_etco2_df = associated_mainstream_etco2_df[
    (associated_mainstream_etco2_df['etCO2'] >= allowed_etco2_range[0]) & (
                associated_mainstream_etco2_df['etCO2'] <= allowed_etco2_range[1])]

## Univariate subgroup analysis

In [ ]:
associated_ptio2_df = associated_ptio2_df.merge(registry_df[['Pat. Nr.', 'Diagnose']], how='left', left_on='pat_nr', right_on='Pat. Nr.')

In [ ]:
associated_ptio2_df['pre_post'] = 'pre'
associated_ptio2_df.loc[associated_ptio2_df['relative_datetime'] > 0, 'pre_post'] = 'post'

In [ ]:
# TBI subgroup
tbi_associated_ptio2_df = associated_ptio2_df[associated_ptio2_df['Diagnose'] == 'TBI']
# compare pre / post ptio2 with mixed effect model
tbi_mixed_model = smf.mixedlm("ptio2 ~ C(pre_post, Treatment(reference='pre'))", tbi_associated_ptio2_df, groups=tbi_associated_ptio2_df['pat_nr'])
tbi_mdf = tbi_mixed_model.fit()
# print formula
print(tbi_mdf.model.formula)
print(tbi_mdf.summary())

In [ ]:
# Stroke subgroup
stroke_associated_ptio2_df = associated_ptio2_df[associated_ptio2_df['Diagnose'] == 'Stroke']
# compare pre / post ptio2 with mixed effect model
stroke_mixed_model = smf.mixedlm("ptio2 ~ C(pre_post, Treatment(reference='pre'))", stroke_associated_ptio2_df, groups=stroke_associated_ptio2_df['pat_nr'])
stroke_mdf = stroke_mixed_model.fit()
# print formula
print(stroke_mdf.model.formula)
print(stroke_mdf.summary())

In [ ]:
key = "C(pre_post, Treatment(reference='pre'))[T.post]"

tbi_coef = tbi_mdf.params[key]
tbi_ci = tbi_mdf.conf_int().loc[key]

stroke_coef = stroke_mdf.params[key]
stroke_ci = stroke_mdf.conf_int().loc[key]

# Create a dataframe for plotting
forest_df = pd.DataFrame({
    'Group': ['TBI', 'Stroke'],
    'Estimate': [tbi_coef, stroke_coef],
    'CI_lower': [tbi_ci[0], stroke_ci[0]],
    'CI_upper': [tbi_ci[1], stroke_ci[1]]
})

In [ ]:
import matplotlib.pyplot as plt

# Plot
fig, ax = plt.subplots(figsize=(6, 4))

# Plot the points and confidence intervals
ax.errorbar(forest_df['Estimate'], forest_df['Group'],
            xerr=[forest_df['Estimate'] - forest_df['CI_lower'], forest_df['CI_upper'] - forest_df['Estimate']],
            fmt='o', capsize=5)

# Add a vertical line at 0 (no effect)
ax.axvline(x=0, linestyle='--', color='gray')

# Labels and title
ax.set_xlabel('Effect of pre_post (post vs pre)')
ax.set_title('Forest Plot of pre_post Effect by Diagnosis Group')

plt.tight_layout()
plt.show()

## Multivariable analysis

In [ ]:
for var_df in [associated_ptio2_df, associated_cpp_df, associated_temperature_df, associated_hr_df]:
    # round to 2 decimals of an hour
    var_df['relative_datetime_cat'] = var_df['relative_datetime'].round(2)

In [ ]:
grouped_ptio2 = associated_ptio2_df.groupby(['pat_nr', 'drug_start', 'relative_datetime_cat']).agg({'ptio2': 'median'}).reset_index()
grouped_cpp = associated_cpp_df.groupby(['pat_nr', 'drug_start', 'relative_datetime_cat']).agg({'cpp': 'median'}).reset_index()
grouped_temperature = associated_temperature_df.groupby(['pat_nr', 'drug_start', 'relative_datetime_cat']).agg({'temperature': 'median'}).reset_index()
grouped_hr = associated_hr_df.groupby(['pat_nr', 'drug_start', 'relative_datetime_cat']).agg({'hr': 'median'}).reset_index()
# grouped_lpr = associated_lpr_df.groupby(['pat_nr', 'drug_start', 'relative_datetime_cat']).agg({'lpr': 'median'}).reset_index()
# grouped_paco2 = associated_paco2_df.groupby(['pat_nr', 'drug_start', 'relative_datetime_cat']).agg({'pCO2_mmHg': 'median'}).reset_index()

In [ ]:
# merge
concatenated_df = grouped_ptio2.merge(grouped_cpp, on=['pat_nr', 'drug_start', 'relative_datetime_cat'], how='outer')
concatenated_df = concatenated_df.merge(grouped_temperature, on=['pat_nr', 'drug_start', 'relative_datetime_cat'], how='outer')
concatenated_df = concatenated_df.merge(grouped_hr, on=['pat_nr', 'drug_start', 'relative_datetime_cat'], how='outer')
# concatenated_df = concatenated_df.merge(grouped_lpr, on=['pat_nr', 'drug_start', 'relative_datetime_cat'], how='outer')
# concatenated_df = concatenated_df.merge(grouped_paco2, on=['pat_nr', 'drug_start', 'relative_datetime_cat'], how='outer')

In [ ]:
concatenated_df['pre_post'] = 'pre'
concatenated_df.loc[concatenated_df['relative_datetime_cat'] >= 0, 'pre_post'] = 'post'

In [ ]:
concatenated_df = concatenated_df.merge(registry_df[['Pat. Nr.', 'Diagnose']], how='left', left_on='pat_nr', right_on='Pat. Nr.')

## Multivariate model
create full mixed effects model with all variables and interactions

In [ ]:
tbi_temp_df = concatenated_df[concatenated_df['Diagnose'] == 'TBI']
# drop rows with nan values
tbi_temp_df = tbi_temp_df.dropna()

tbi_mixed_model = smf.mixedlm("ptio2 ~ C(pre_post, Treatment(reference='pre')) * cpp * temperature * hr", tbi_temp_df, groups=tbi_temp_df['pat_nr'])
tbi_mixed_model_fit = tbi_mixed_model.fit()
print(tbi_mixed_model.score(tbi_mixed_model_fit.params_object))
print(tbi_mixed_model_fit.summary())

In [ ]:
# with centered variables and reducing complexity
for var in ['cpp', 'temperature', 'hr']:
    tbi_temp_df[var + '_c'] = tbi_temp_df[var] - tbi_temp_df[var].mean()

tbi_model = smf.mixedlm(
    "ptio2 ~ C(pre_post, Treatment(reference='pre')) * (cpp_c + temperature_c + hr_c)",
    tbi_temp_df,
    groups=tbi_temp_df['pat_nr']
)

tbi_mixed_model_fit = tbi_mixed_model.fit()
print(tbi_mixed_model.score(tbi_mixed_model_fit.params_object))

print(tbi_mixed_model_fit.summary())


In [ ]:
stroke_temp_df = concatenated_df[concatenated_df['Diagnose'] == 'Stroke']
# drop rows with nan values
stroke_temp_df = stroke_temp_df.dropna()
stroke_mixed_model = smf.mixedlm("ptio2 ~ C(pre_post, Treatment(reference='pre')) * cpp * temperature * hr", stroke_temp_df, groups=stroke_temp_df['pat_nr'])
stroke_mixed_model_fit = stroke_mixed_model.fit()
print(stroke_mixed_model.score(stroke_mixed_model_fit.params_object))
print(stroke_mixed_model_fit.summary())

In [ ]:
# with centered variables and reducing complexity
for var in ['cpp', 'temperature', 'hr']:
    stroke_temp_df[var + '_c'] = stroke_temp_df[var] - stroke_temp_df[var].mean()

stroke_model = smf.mixedlm(
    "ptio2 ~ C(pre_post, Treatment(reference='pre')) * (cpp_c + temperature_c + hr_c)",
    stroke_temp_df,
    groups=stroke_temp_df['pat_nr']
)
stroke_mixed_model_fit = stroke_model.fit()
print(stroke_model.score(stroke_mixed_model_fit.params_object))
print(stroke_mixed_model_fit.summary())

In [ ]:
# plot subgroup forest plot

tbi_coef = tbi_mixed_model_fit.params[key]
tbi_ci = tbi_mixed_model_fit.conf_int().loc[key]
stroke_coef = stroke_mixed_model_fit.params[key]
stroke_ci = stroke_mixed_model_fit.conf_int().loc[key]
# Create a dataframe for plotting
forest_df = pd.DataFrame({
    'Group': ['TBI', 'Stroke'],
    'Estimate': [tbi_coef, stroke_coef],
    'CI_lower': [tbi_ci[0], stroke_ci[0]],
    'CI_upper': [tbi_ci[1], stroke_ci[1]]
})

# Plot
fig, ax = plt.subplots(figsize=(6, 3))
# Plot the points and confidence intervals
ax.errorbar(forest_df['Estimate'], forest_df['Group'],
            xerr=[forest_df['Estimate'] - forest_df['CI_lower'], forest_df['CI_upper'] - forest_df['Estimate']],
            fmt='o', capsize=5)
# Add a vertical line at 0 (no effect)
ax.axvline(x=0, linestyle='--', color='gray')
# Labels and title
ax.set_xlabel('Effect of pre_post (post vs pre)')
ax.set_title('Forest Plot of pre_post Effect by Diagnosis Group')

ax.set_xlim(-3, 3)

# add space above and below the categories and decrease space between the categories
ax.set_ylim(-0.5, 1.5)
# and decrease space between the categories
ax.set_yticks([0, 1])
ax.set_yticklabels(['TBI', 'Stroke'])

plt.tight_layout()
plt.show()

# Plot effect

In [ ]:
print('TBI:')
print(
    f'Pre:  {concatenated_df[(concatenated_df["Diagnose"] == "TBI" ) & (concatenated_df["pre_post"] == "pre")]["ptio2"].median()} ({concatenated_df[(concatenated_df["Diagnose"] == "TBI") & (concatenated_df["pre_post"] == "pre")]["ptio2"].quantile(0.25)}, {concatenated_df[(concatenated_df["Diagnose"] == "TBI") & (concatenated_df["pre_post"] == "pre")]["ptio2"].quantile(0.75)})',
    f'Post: {concatenated_df[(concatenated_df["Diagnose"] == "TBI") & (concatenated_df["pre_post"] == "post")]["ptio2"].median()} ({concatenated_df[(concatenated_df["Diagnose"] == "TBI") & (concatenated_df["pre_post"] == "post")]["ptio2"].quantile(0.25)}, {concatenated_df[(concatenated_df["Diagnose"] == "TBI") & (concatenated_df["pre_post"] == "post")]["ptio2"].quantile(0.75)})')
print(f'p-value: {tbi_mixed_model_fit.pvalues[key]}')

print('Stroke:')
print(
    f'Pre:  {concatenated_df[(concatenated_df["Diagnose"] == "Stroke") & (concatenated_df["pre_post"] == "pre")]["ptio2"].median()} ({concatenated_df[(concatenated_df["Diagnose"] == "Stroke") & (concatenated_df["pre_post"] == "pre")]["ptio2"].quantile(0.25)}, {concatenated_df[(concatenated_df["Diagnose"] == "Stroke") & (concatenated_df["pre_post"] == "pre")]["ptio2"].quantile(0.75)})',
    f'Post: {concatenated_df[(concatenated_df["Diagnose"] == "Stroke") & (concatenated_df["pre_post"] == "post")]["ptio2"].median()} ({concatenated_df[(concatenated_df["Diagnose"] == "Stroke") & (concatenated_df["pre_post"] == "post")]["ptio2"].quantile(0.25)}, {concatenated_df[(concatenated_df["Diagnose"] == "Stroke") & (concatenated_df["pre_post"] == "post")]["ptio2"].quantile(0.75)})')
print(f'p-value: {stroke_mixed_model_fit.pvalues[key]}')

In [ ]:
# Plot the distribution of ptio2 values for each group in pre and post
ax = sns.boxplot(data=concatenated_df, x='Diagnose', y='ptio2', hue='pre_post', showfliers=False)
ax.set_xlabel('')
ax.set_ylabel('PtO2 (mmHg)')

# set legend title
ax.legend(title='', loc='upper right')

plt.show()

# Single variable models

paco2